In [21]:
import pandas as pd
import csv
import glob
import os
from datetime import date

today = date.today()

In [22]:
home_page = "[The Turkey-Palestine Source Collection](turkey_palestine_collection.md)"
separator = " // "

page_heading_element = "# "
page_heading_element_2 = "## "

project_folder = 'tpc\Sources Database'
upper_category = 'firstlevel_'

## Import Files

In [23]:
articles_view = pd.read_csv(rf'C:\Users\act1780\Documents\GitHub\ACortellari.github.io\{project_folder}\articles_view.csv', delimiter='\t')
publications_view = pd.read_csv(fr'C:\Users\act1780\Documents\GitHub\ACortellari.github.io\{project_folder}\publications_view.csv', delimiter='\t')
issues_table = pd.read_csv(fr'C:\Users\act1780\Documents\GitHub\ACortellari.github.io\{project_folder}\issues_view.csv', delimiter='\t')

Create a Last Update Box

In [24]:
last_update = f"""
```
Last Update: {today}
```
"""

Create a list of available views from the database

In [25]:
folder_path = fr"C:\Users\act1780\Documents\GitHub\ACortellari.github.io\{project_folder}/*" 
files_list = glob.glob(folder_path)

list_of_files = []
for file_path in files_list:
    file_path = file_path.split('.')[-2].split("\\")[-1]
    file_path = f"* {file_path}"
    list_of_files.append(file_path)

files = str(list_of_files).replace("'", "").replace("[", "").replace("]", "").replace(",", "")
files

'* articles_view * issues_view * publications_view'

## Create Main Publications Page

In [26]:
publications_view['publication_title'] = publications_view.apply(lambda row: f"[{row['publication_title']}](publications_{row['publication_title'].replace(' ', '_')}.md)", axis=1)
metadata = f"In our dataset, we have {len(publications_view['publication_title'])} unique publication names. The earliest publication started in {min(publications_view['first_year'])}, while the latest publication ended in {max(publications_view['last_year'])}. These publications are spread across various locations including {', '.join(publications_view['publication_location'].unique())}."

filename = rf"{upper_category}publications.md"
with open(filename, 'w') as f:
    f.write(f"{page_heading_element}{home_page}{separator}Publications\n\n")
    f.write(f"{metadata}\n\n")
    f.write((publications_view).to_markdown(index=False))

## Create each Publications' Page

In [27]:
columns_publications_articles = ['article_id', 'article_text', 'publication_title']

In [28]:
csv_file = fr'C:\Users\act1780\Documents\GitHub\ACortellari.github.io\{project_folder}\publications_view.csv'  # Change this to the path of your CSV file
    
with open(csv_file, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, delimiter='\t')
        print(reader)
        for row in reader:
            publication_name = row['publication_title']
            publication_type = row['publication_type']
            start_year = row['first_year']
            end_year = row['last_year']
            total_issues = row['total_issues']
            publication_location = row['publication_location']

            metadata = f"{publication_name} was a {publication_type} publication. It published {total_issues} issues in {publication_location} between {start_year} and {end_year}."
            
            filename = rf"publications_{publication_name.replace(' ', '_')}.md"
            with open(filename, 'w') as f:
                f.write(f"{page_heading_element}[Publications]({upper_category}publications.md){separator}{publication_name}\n\n")
                f.write(f"{metadata}\n\n")
                f.write(f"{page_heading_element_2}Articles in Scope\n\n")
                f.write(f"{articles_view[articles_view['publication_title'] == publication_name].drop(columns=columns_publications_articles).to_markdown(index=False)}\n\n")
                f.write(f"{page_heading_element_2}Issues Summary\n\n")
                f.write(issues_table[issues_table['publication_title'] == publication_name].sort_values(by=['issue_year', 'issue_month']).to_markdown(index=False))

## Main Article Page

In [29]:
filename = rf"{upper_category}articles.md"
with open(filename, 'w') as f:
    f.write(f"{page_heading_element}{home_page}{separator}Articles\n\n")
    f.write((articles_view.drop(columns=['article_id', 'article_text'])).to_markdown(index=False))

# Create each articles' page

In [30]:
articles_with_text = articles_view[articles_view['article_text'].notna()]

for index, row in articles_with_text.iterrows():
    filename = rf"articles_{row['article_title'].replace('?', '')}.md"
    metadata_table = articles_with_text.loc[articles_with_text['article_title'] == row['article_title']].drop(columns={'article_text'})

    with open(filename, 'w') as f:
        f.write(f"{page_heading_element}{row['article_title']}\n\n")
        f.write(f"{metadata_table.to_markdown(index=False)}\n\n")
        #f.write(f"**Author:** {row['author']}\n\n")
        #f.write(f"**Published on:** [{row['publication_title']}]({row['publication_title']}.md)\n\n")
        #f.write(f"**Issue:** {row['issue_number']}, {row['issue_date']}\n\n")
        #f.write(f"**Pages:** {row['pages']}\n\n")
        f.write(f"{row['article_text']}\n")

## Create the Collection main page

### Create a main menu

In [31]:
folder_path = r"C:\Users\act1780\Documents\GitHub\ACortellari.github.io\tpc/first*.md"  # Replace this with the path to your folder
first_level_list = glob.glob(folder_path)

menu = f'{separator}'.join([f"[{levels.split(os.path.sep)[-1].split('_')[-1].split('.')[-2].upper()}]({levels.split(os.path.sep)[-1]})" for levels in first_level_list])
menu

'[ARTICLES](firstlevel_articles.md) // [PUBLICATIONS](firstlevel_publications.md)'

In [32]:
filename = rf"turkey_palestine_collection.md"
with open(filename, 'w') as f:
    f.write(f"{page_heading_element}{home_page}\n\n")
    f.write(f"{page_heading_element}{menu}")
    f.write(f"{last_update}\n\n")
    f.write(f"This page is a humble list of sources, primarily collected from mainstream and leftist press, that can be useful to those interested in studying the history of the radical left in Turkey, specifically its linkages and intellectual connections with Palestinian organizations from the late 1960s to the early 1970s.\n\n") 
    f.write(f"The bulk of the corpus listed—and sometimes provided in textual form—here is the result of research efforts that occupied me in 2018-19 while working on the Master's Thesis as part of my MA in Middle Eastern Studies at Leiden University. I completed my thesis, titled <a href='https://studenttheses.universiteitleiden.nl/handle/1887/82728'>Bringing Palestine Home: A Transnational History of Turkey's Radical Left and Palestine (1967-1972)</a>, under the supervision of Dr. Alp A. Yenen.\n\n") 
    f.write(f"The rich corpus of primary sources accessible through the digitization efforts of Türkiye Sosyal Tarih Araştırma Vakfı (TÜSTAV) significantly facilitated my research. I am grateful for TÜSTAV's valuable contributions to my endeavor. If any user believes that a valuable source is missing from this list (and that is most likely the case), I welcome and encourage suggestions. Your input is invaluable in enhancing the comprehensiveness of this small collection.\n\n")
    f.write(f"{page_heading_element_2}Sources in the Leftist Press\n\n")
    f.write(f"The sources provided here span the years 1968 to 1971 representing just a fraction of the leftist weekly and monthly publications from that period. Although the initial corpus was more extensive, only the sources included in my bibliography are currently listed. Additional sources may be made available here in the future. Sources published in leftist publications will gradually be made available in the <a href='https://github.com/ACortellari/ACortellari.github.io/tree/main/Turkey%20Palestine%20Archive/Sources%20Database'>Sources Database</a> (as .csv files for ease in fruition).\n\n") 
    f.write(f"The database includes the following:\n\n")
    f.write("\n".join([file_path for file_path in list_of_files]))